<a href="https://colab.research.google.com/github/vpysmennyi/machinelearning-learning/blob/main/Lection5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline

import numpy as np
import torch
import matplotlib.pyplot as plt
import torchvision

from torch import nn
from torch.utils.data import DataLoader
from torch.optim import SGD


In [9]:
train_ds = torchvision.datasets.MNIST('.', train=True, download=True, transform=torchvision.transforms.ToTensor())
val_ds = torchvision.datasets.MNIST('.', train=False, download=True, transform=torchvision.transforms.ToTensor())

In [10]:
batch_size = 128
hidden_size = 128
train_dl = DataLoader(train_ds, batch_size = batch_size, shuffle=True)
val_dl = DataLoader(train_ds, batch_size = batch_size)

In [12]:
w1 = nn.Parameter(torch.Tensor(784,hidden_size))
b1 = nn.Parameter(torch.Tensor(hidden_size)) #bias

w2 = nn.Parameter(torch.Tensor(hidden_size,hidden_size))
b2 = nn.Parameter(torch.Tensor(hidden_size))

w3 = nn.Parameter(torch.Tensor(hidden_size,10))
b3 = nn.Parameter(torch.Tensor(10))

# adjusted normal initialization
#nn.init.normal_(w1, std=np.sqrt(1. / 784))
#nn.init.normal_(w2, std=np.sqrt(1. / hidden_size))
#nn.init.normal_(w3, std=np.sqrt(1. / hidden_size))

#Xavier uniform initialization
#p = np.sqrt(6)/ np.sqrt(784 + hidden_size)
#nn.init.uniform_(w1, -p, p)

#p = np.sqrt(6)/ np.sqrt(hidden_size + hidden_size)
#nn.init.uniform_(w2, -p, p)

#p = np.sqrt(6)/ np.sqrt(hidden_size + 10)
#nn.init.uniform_(w3, -p, p)

#Kaiming normal initialization
nn.init.normal_(w1, std=np.sqrt(2. / 784))
nn.init.normal_(w2, std=np.sqrt(2. / hidden_size))
nn.init.normal_(w3, std=np.sqrt(2. / hidden_size))

nn.init.zeros_(b1)
nn.init.zeros_(b2)
nn.init.zeros_(b3)

loss_fn = nn.NLLLoss()
opt = SGD([w1,b1,w2,b2,w3,b3], lr = 0.1)

print(w1.mean(), w1.std())
n_epoch = 10

for epoch_ind in range(n_epoch):
  epoch_loss = []
  epoch_accuracy = []

  for batch in train_dl:
    opt.zero_grad() #zerofy all gradients, new graph created
    X,Y = batch
    X = X.reshape((X.shape[0], -1))
    #print(X.shape)

    y1 = torch.relu(X @ w1 + b1) 
    y2 = torch.relu(y1 @ w2 + b2)
    Yp = torch.softmax(y2 @ w3 + b3, dim=1) #softmax used for multiclass regression, not using sigmoid
    
    loss = loss_fn(Yp,Y)
    loss.backward()

    opt.step() # w1 = w1 - Dw1 : update weights

    epoch_loss.append(loss.detach().cpu().numpy())

    accuracy = (Yp.argmax(1) == Y).sum()/len(Y)
    epoch_accuracy.append(accuracy)
    #print(batch[0].shape, batch[1].shape)
    #print(loss)

  print(f"Epoch  {epoch_ind}: Loss: {np.mean(epoch_loss):.6f}\t Acc={np.mean(epoch_accuracy):.4f}")

  for batch in val_dl:
    X,Y = batch
    X = X.reshape((X.shape[0], -1))
    
    y1 = torch.relu(X @ w1 + b1) 
    y2 = torch.relu(y1 @ w2 + b2)
    Yp = torch.softmax(y2 @ w3 + b3, dim=1)

    loss = loss_fn(Yp,Y)
    epoch_loss.append(loss.detach().cpu().numpy())

    accuracy = (Yp.argmax(1) == Y).sum()/len(Y)
    epoch_accuracy.append(accuracy)

  print(f"Val Epoch  {epoch_ind}: Loss: {np.mean(epoch_loss):.6f}\t Acc={np.mean(epoch_accuracy):.4f}")

tensor(-0.0002, grad_fn=<MeanBackward0>) tensor(0.0505, grad_fn=<StdBackward0>)
Epoch  0: Loss: -0.544045	 Acc=0.5936
Val Epoch  0: Loss: -0.628294	 Acc=0.6649
Epoch  1: Loss: -0.789051	 Acc=0.8158
Val Epoch  1: Loss: -0.799928	 Acc=0.8220
Epoch  2: Loss: -0.819479	 Acc=0.8336
Val Epoch  2: Loss: -0.822894	 Acc=0.8365
Epoch  3: Loss: -0.830314	 Acc=0.8414
Val Epoch  3: Loss: -0.832722	 Acc=0.8431
Epoch  4: Loss: -0.837173	 Acc=0.8463
Val Epoch  4: Loss: -0.838803	 Acc=0.8476
Epoch  5: Loss: -0.842327	 Acc=0.8500
Val Epoch  5: Loss: -0.843551	 Acc=0.8514
Epoch  6: Loss: -0.846765	 Acc=0.8541
Val Epoch  6: Loss: -0.847011	 Acc=0.8545
Epoch  7: Loss: -0.850076	 Acc=0.8571
Val Epoch  7: Loss: -0.851722	 Acc=0.8587
Epoch  8: Loss: -0.853433	 Acc=0.8601
Val Epoch  8: Loss: -0.854416	 Acc=0.8609
Epoch  9: Loss: -0.856147	 Acc=0.8623
Val Epoch  9: Loss: -0.857182	 Acc=0.8633
